[야놀자 리뷰 크롤링]
-지역: 제주 
-필요한 데이터: 별점 4점 이상의 전체 호텔에 대한 리뷰

In [ ]:
import requests
from bs4 import BeautifulSoup

import json

from tqdm.notebook import tqdm

import pandas as pd
import datetime
import numpy as np

In [ ]:
# 전체 호텔에 대한 호텔 ID 가져오기 
url = "https://www.yanolja.com/api/v1/contents/search?"
last_page=19

# 호텔 초기화
# id="호텔ID", addr="주소", name="호텔이름", grade="등급"
hotel_ids, addrs, names, grades = [],[],[],[]
hotel={
    "id": hotel_ids, 
    "name":names,
    "grade" :grades,
    "addr":addrs, 
    "site" :"야놀자"
}

for page in tqdm(range(1,last_page)):
  today = datetime.date.today()
  params= {
      "advert":"KEYWORD",
      "capacityAdult":2,
      "capacityChild":0,
      "checkinDate" : datetime.date.today(),
      "checkoutDate" :(today + datetime.timedelta(days=1)).strftime("%Y-%m-%d"),
      "hotel" :"1",
      "keyword":"제주",
      "lat":37.50681,
      "lng":127.06624,
      "page":{page},
      "limit":"20",
      "searchKeyword":"제주",
      "searchType" :"hotel",
      "sort":100,
      "pathDivision":"keyword-hotel",
      "themes":""

  }

  headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
             "X-Requested-With": "XMLHttpRequest"
            }

  resp = requests.get(url, params=params, headers=headers)
  dict_resp = resp.json()
  motel_list = dict_resp["motels"]["lists"]

  for indx in range(len(motel_list)):
    hotel_ids.append(motel_list[indx]["key"])
    names.append(motel_list[indx]["name"])
    grades.append(motel_list[indx]["gradeInfo"]["title"])
    addrs.append(motel_list[indx]["addr1"])
  

  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
# 총 호텔의 갯수
len(hotel["id"])

351

In [ ]:
dic_id_addrs = dict(zip(hotel_ids, addrs))

In [ ]:
#야놀자 호텔만 처리해주는 부분
dic_id_addrs['1000114073'] ='제주특별자치도 서귀포시 색달동 2812-2'
dic_id_addrs['10043138'] ='제주특별자치도 제주시 한림읍 협재리 1193'
dic_id_addrs['10042831'] ='제주특별자치도 서귀포시 대정읍 안성리 214'
dic_id_addrs['3001179'] ='제주특별자치도 서귀포시 회수동 30 위(WE)호텔'
dic_id_addrs['10043764'] ='제주특별자치도 제주시 우도면 연평리 1707'
dic_id_addrs['3001485'] ='제주특별자치도 서귀포시 하예동 1888 베이힐풀&빌라'
dic_id_addrs['10041791'] ='제주특별자치도 서귀포시 안덕면 화순리 1066-3'

In [ ]:
# 주소 값 지번 주소로 통일 
import re

chanage_addrs=[]

for ad in tqdm(dic_id_addrs.values()):
  url = 'https://www.juso.go.kr/support/AddressMainSearch.do'
  params = {
    'searchKeyword': ad
  }
  resp = requests.get(url, params = params)
  soup1 = BeautifulSoup(resp.content, 'lxml')
  jiban = soup1.select('div.subejct_2 span.roadNameText')
  
  chanage_addrs.append(jiban[0].text.replace('\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t','').strip('\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t'))


  0%|          | 0/351 [00:00<?, ?it/s]

In [ ]:
# 바뀐 주소 설정 해준 hotel 정보 
hotel_info ={
    "id": hotel["id"], 
    "name":hotel["name"],
    "grade" :hotel['grade'],
    "addr":chanage_addrs, 
    "site" :hotel['site']
}
hotel_info 


{'addr': ['제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 제주시 연동  290-1 성남빌라',
  '제주특별자치도 제주시 용담삼동  2555 ciel bleu 호텔',
  '제주특별자치도 제주시 연동  274-16 SHILLA STAY JEJU HOTEL',
  '제주특별자치도 제주시 외도일동  401-8',
  '제주특별자치도 제주시 건입동  1443 호텔리젠트마린더블루',
  '제주특별자치도 서귀포시 서귀동  182 글램블루호텔',
  '제주특별자치도 제주시 해안동  2101-3',
  '제주특별자치도 서귀포시 강정동  173-1',
  '제주특별자치도 서귀포시 서귀동  314-2',
  '제주특별자치도 제주시 연동  2334-4',
  '제주특별자치도 서귀포시 상예동  625',
  '제주특별자치도 제주시 이도이동  1025-8',
  '제주특별자치도 제주시 연동  263-2 제주1번가 더테라스',
  '제주특별자치도 서귀포시 성산읍 시흥리 200 더포그레이스호텔',
  '제주특별자치도 제주시 노형동  925 제주 드림타워 복합리조트',
  '제주특별자치도 서귀포시 성산읍 고성리 1201 디아일랜드 마리나 호텔',
  '제주특별자치도 제주시 조천읍 함덕리 1269-9 유탑유블레스호텔',
  '제주특별자치도 서귀포시 서호동  1522-1',
  '제주특별자치도 서귀포시 강정동  173 호텔빠레브',
  '제주특별자치도 제주시 건입동  1443-2 호텔 휘슬',
  '제주특별자치도 서귀포시 안덕면 서광리 산24',
  '제주특별자치도 제주시 연동  263-15 메종글래드 제주',
  '제주특별자치도 제주시 연동  251-1 엠버서더 관광호텔',
  '제주특별자치도 제주시 연동  281-22',
  '제주특별자치도 제주시 연동  2324-6 제주연동 롯데시티호텔',
  '제주특별자치도 서귀포시 성산읍 시흥리 208 봄그리고가을리조트',
  '제주특별자치도 서귀포시 중문동  2700-2 부영호텔',
  '제주특별자치도 서귀포시 서귀동

In [ ]:
df_hotel_info = pd.DataFrame(hotel_info)
df_hotel_info

,id,name,grade,addr,site
0,1000111292,타마라 호텔 제주,일반호텔,제주특별자치도 서귀포시 법환동 745-2,야놀자
1,1000112785,벤티모 호텔 앤 레지던스 제주,레지던스,제주특별자치도 제주시 연동 290-1 성남빌라,야놀자
2,1000110848,팜파스호텔 제주,3성급,제주특별자치도 제주시 용담삼동 2555 ciel bleu 호텔,야놀자
3,3001182,신라스테이 제주,4성급,제주특별자치도 제주시 연동 274-16 SHILLA STAY JEJU HOTEL,야놀자
4,3001098,그라벨 호텔,4성급,제주특별자치도 제주시 외도일동 401-8,야놀자
...,...,...,...,...,...
346,1000111493,뉴 아일랜드 제주,일반호텔,제주특별자치도 제주시 연동 263-12 뉴아일랜드관광호텔,야놀자
347,1000103550,제주 나비스호텔,일반호텔,제주특별자치도 제주시 애월읍 하귀2리 2749-3 나비스호텔,야놀자
348,10045511,드림 제주 호텔,일반호텔,제주특별자치도 서귀포시 대정읍 하모리 2132,야놀자
349,10045638,제주 하루시다,일반호텔,제주특별자치도 서귀포시 안덕면 사계리 724,야놀자


In [ ]:
df_hotel_info.to_csv('hotel_info.csv', index=False, encoding="utf-8-sig")

In [ ]:
# 전체 호텔의 리뷰 들고오기 
hotel_review={
    "name" :[],
    "review":[],
    "score":[],
    "addr":[],
    "site":"야놀자"
}
for indx in tqdm(range(len(hotel_ids))): 
  url =f'https://place-site.yanolja.com/api/stay.properties.review-list.get?batch=1&input=%7B%220%22%3A%7B%22propertyId%22%3A{hotel_ids[indx]}%2C%22query%22%3A%7B%22photoReviewOnly%22%3Afalse%2C%22roomTypeIds%22%3A%22%22%2C%22page%22%3A1%2C%22sort%22%3A%22BEST%22%7D%2C%22cursor%22%3A1%7D%7D'

  headers = {
      "Referer":f'https://place-site.yanolja.com/places/{hotel_ids[indx]}',
      "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
      'Cookie' : '_gcl_au=1.1.1544913891.1658213121; _fbp=fb.1.1658213121690.650429316; cgntId=ap-northeast-2:6e2ea3cc-10d2-4cac-bfce-6cace99eee87; _gid=GA1.2.1564562568.1658392771; AUFAH5A41667571241=1658242800000000000|4|1658392770481730276|1|1658140565353646299; _ga=GA1.2.1201862374.1658210909; _ga_8KQT8SBL6R=GS1.1.1658401676.10.0.1658401676.60; yanolja_sid=s%3ALK5a1wVQ8mYUwRkndmM-MsVIx99ly3rk.DzXoHbi5YTQcGYizwwEphzWunyebtZb6wPj2mZli%2FdY; ab.storage.deviceId.f4231aff-9421-403b-90b6-6b5aa13a4ef8=%7B%22g%22%3A%22842aad22-30e1-7c74-5795-05c6033ab0c8%22%2C%22c%22%3A1658213142050%2C%22l%22%3A1658456074725%7D; ab.storage.userId.f4231aff-9421-403b-90b6-6b5aa13a4ef8=%7B%22g%22%3A%22ap-northeast-2%3A6e2ea3cc-10d2-4cac-bfce-6cace99eee87%22%2C%22c%22%3A1658274878399%2C%22l%22%3A1658456074726%7D; _gat_gtag_UA_87400434_10=1; ab.storage.sessionId.f4231aff-9421-403b-90b6-6b5aa13a4ef8=%7B%22g%22%3A%2297fafec5-6ed4-0bbc-aff1-4ead03939d85%22%2C%22e%22%3A1658457880873%2C%22c%22%3A1658456074723%2C%22l%22%3A1658456080873%7D'
  }
  resp = requests.get(url, headers=headers)
  comment = json.loads(resp.text)
  page_count = comment[0]['result']['data']['meta']['reviewCount']
  total_page = int(np.ceil(page_count/20))

  
  for no in tqdm(range(1, total_page+1)):
    page_url =f'https://place-site.yanolja.com/api/stay.properties.review-list.get?batch=1&input=%7B%220%22%3A%7B%22propertyId%22%3A{hotel_ids[indx]}%2C%22query%22%3A%7B%22photoReviewOnly%22%3Afalse%2C%22roomTypeIds%22%3A%22%22%2C%22page%22%3A1%2C%22sort%22%3A%22BEST%22%7D%2C%22cursor%22%3A{no}%7D%7D'

    resp = requests.get(url=page_url,  headers=headers)
    resp.encoding = 'utf-8'
    js = json.loads(resp.text)

    comment= js[0]['result']['data']['reviews']
   
    for i in range(len(comment)):
      review_score = comment[i]["score"]
      if review_score >=4:
        hotel_review["name"].append(hotel_info["name"][indx])
        hotel_review["addr"].append(hotel_info["addr"][indx])
        hotel_review['review'].append(comment[i]['content'])


hotel_review

  0%|          | 0/351 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'addr': ['제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특별자치도 서귀포시 법환동  745-2',
  '제주특

In [ ]:
import pandas as pd

df_yanolja_hotel = pd.DataFrame(hotel_review)

df_yanolja_hotel

,name,review,score,addr,site
0,타마라 호텔 제주,[신축이라 깨끗하고 깔끔해서 좋았어요. 불필요한 건 뺀 대신 필요한 것만 있는 진짜...,5,제주특별자치도 서귀포시 법환동 745-2,야놀자
1,타마라 호텔 제주,[이가격에 이숙소 실화인가요??이틀동안 너무나도 잘쉬고갑니다. 위치도 너무 좋고 신...,5,제주특별자치도 서귀포시 법환동 745-2,야놀자
2,타마라 호텔 제주,[완전 친절하시고 객실 완전 청결해요. 다음에 제주도 올 때 또 여기서 머물거예요!...,5,제주특별자치도 서귀포시 법환동 745-2,야놀자
3,타마라 호텔 제주,[편히 쉬다갑니다!!],5,제주특별자치도 서귀포시 법환동 745-2,야놀자
4,타마라 호텔 제주,"[조금좁다고생각하면 좁을수있지만 그냥 여기하셔, 이가격에 여기보다 좋은곳 없어유, ...",5,제주특별자치도 서귀포시 법환동 745-2,야놀자
...,...,...,...,...,...
98221,제주 나비스호텔,[편안히 잘 머무르다 갑니다 직원분도 친절하셨어요 감사합니다],5,제주특별자치도 제주시 애월읍 하귀2리 2749-3 나비스호텔,야놀자
98222,제주 슬림호텔,[언제나 좋아요],5,제주특별자치도 제주시 이도이동 1776-31,야놀자
98223,제주 슬림호텔,[좋았습니다],5,제주특별자치도 제주시 이도이동 1776-31,야놀자
98224,제주 슬림호텔,[좋고 우ㅣ치 좋았어요],5,제주특별자치도 제주시 이도이동 1776-31,야놀자


In [ ]:
#야놀자 소스 csv파일로 만들기  
df_yanolja_hotel.to_csv('야놀자.csv', index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd

df_yanolja_hotel_new = pd.DataFrame(new_hotel_info)

df_yanolja_hotel_new

,id,name,grade,addr,site
0,1000111292,타마라 호텔 제주,일반호텔,제주특별자치도 서귀포시 법환동 745-2,야놀자
1,1000112785,벤티모 호텔 앤 레지던스 제주,레지던스,제주특별자치도 제주시 연동 290-1 성남빌라,야놀자
2,1000110848,팜파스호텔 제주,3성급,제주특별자치도 제주시 용담삼동 2555 ciel bleu 호텔,야놀자
3,3001182,신라스테이 제주,4성급,제주특별자치도 제주시 연동 274-16 SHILLA STAY JEJU HOTEL,야놀자
4,3001098,그라벨 호텔,4성급,제주특별자치도 제주시 외도일동 401-8,야놀자
...,...,...,...,...,...
346,1000111493,뉴 아일랜드 제주,일반호텔,제주특별자치도 제주시 연동 263-12 뉴아일랜드관광호텔,야놀자
347,1000103550,제주 나비스호텔,일반호텔,제주특별자치도 제주시 애월읍 하귀2리 2749-3 나비스호텔,야놀자
348,10045511,드림 제주 호텔,일반호텔,제주특별자치도 서귀포시 대정읍 하모리 2132,야놀자
349,10045638,제주 하루시다,일반호텔,제주특별자치도 서귀포시 안덕면 사계리 724,야놀자
